In [1]:
import pandas as pd
import numpy as np
import sys
import jieba
import os
import pickle   #持久化
from numpy import *
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer #TF-IDF向量转换类
from sklearn.feature_extraction.text import TfidfVectorizer #TF_IDF向量生成类
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer 
from sklearn.datasets.base import Bunch
from sklearn.naive_bayes import MultinomialNB #多项式贝叶斯算法
import gc #解決memory error問題
from sklearn.utils import shuffle
import random
import gensim
from gensim.models import word2vec
from gensim.models import doc2vec
import pickle
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import np_utils
from tensorflow.keras import Sequential

Using TensorFlow backend.


In [2]:
with open('toutiao_cat_data.txt', 'r', encoding='utf8') as f:
    data = f.read().splitlines()

In [3]:
data

['\ufeff6551700932705387022_!_101_!_news_culture_!_京城最值得你来场文化之旅的博物馆_!_保利集团,马未都,中国科学技术馆,博物馆,新中国',
 '6552368441838272771_!_101_!_news_culture_!_发酵床的垫料种类有哪些？哪种更好？_!_',
 '6552407965343678723_!_101_!_news_culture_!_上联：黄山黄河黄皮肤黄土高原。怎么对下联？_!_',
 '6552332417753940238_!_101_!_news_culture_!_林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？_!_',
 '6552475601595269390_!_101_!_news_culture_!_黄杨木是什么树？_!_',
 '6552387648126714125_!_101_!_news_culture_!_上联：草根登上星光道，怎么对下联？_!_',
 '6552271725814350087_!_101_!_news_culture_!_什么是超写实绘画？_!_',
 '6552452982015787268_!_101_!_news_culture_!_松涛听雨莺婉转，下联？_!_',
 '6552400379030536455_!_101_!_news_culture_!_上联：老子骑牛读书，下联怎么对？_!_',
 '6552339283632455939_!_101_!_news_culture_!_上联：山水醉人何须酒。如何对下联？_!_',
 '6552387367334838792_!_101_!_news_culture_!_国画山水，如何读懂山水画_!_林风眠,黄海归来步步云,秋山图,计白当黑,山水画,江山万里图,张大千,巫峡清秋图,活眼,山雨欲来图',
 '6552314684383429128_!_101_!_news_culture_!_一元硬币是这种，现在价值24000元，找找看！_!_牡丹,收藏价值',
 '6552128476109865229_!_101_!_news_culture_!_有哪些让人感动的语句呢？_!_',
 '6552447749072093453_!_101_!_news_culture_!_上联

In [4]:
label = []
title = []
keyword = []

for i in range(len(data)):
    label.append(data[i].split('_!_')[1])
    title.append(data[i].split('_!_')[3])
    keyword.append(data[i].split('_!_')[4])

In [5]:
keyword = pd.DataFrame(keyword)
title = pd.DataFrame(title)
label = pd.DataFrame(label)
trainData = pd.concat( [title , keyword], axis=1 )
trainData = pd.concat( [trainData , label], axis=1 )
trainData.columns = ['title', 'keyword', 'label']
trainData = shuffle(trainData)

# trainData = trainData[:20000]
trainData.index = range(len(trainData)) # row index從0開始
trainData = trainData.drop(['keyword'], axis=1)
trainData.head()

,title,label
0,余额宝升级后，新基金的收益的流动性如何？,109
1,马云马化腾给这位部级当副手,109
2,慢跑对身体有什么好处？,103
3,特斯拉再出车祸，碰撞起火二死一伤，自动驾驶安全有保障吗？,109
4,李沁出席活动，网友：穿单肩裙子就是美！,102


In [6]:
# #解決memory error問題，釋放記憶體
# del label
del title
del keyword
gc.collect()

48

In [7]:
#停用詞
stopwords=[]
for word in open('stopwords.txt','r', encoding='utf8'):
    stopwords.append(word.strip())

In [8]:
print(sys.getdefaultencoding())

utf-8


In [9]:
title_file = open("title_split.txt", "w", encoding = 'utf8')

#分詞(title和keyword)
splitTitle = []
for i in range(len(trainData.title)):
    split = jieba.cut(trainData.title[i], cut_all=False)
    temp = ""
    for word in split:
        if word not in stopwords:  #若word不在停用詞裡面，則加入
            temp = temp + word + " "
    temp = temp.strip()  #刪除不必要的空白
    
    title_file.writelines(str(temp + "\n"))   #存檔(分割詞，已扣除stopword)
    splitTitle.append(temp)

title_file.close()
    
# splitKeyword = []
# for i in range(len(trainData.keyword)):
#     split = jieba.cut(trainData.keyword[i], cut_all=False)
#     temp = ""
#     for word in split:
#         if word not in stopwords:  #若word不在停用詞裡面，則加入
#             temp = temp + word + " "   
#     temp = temp.strip()  #刪除不必要的空白
#     splitKeyword.append(temp)

Building prefix dict from the default dictionary ...
I0802 13:37:09.727056 15972 __init__.py:111] Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\jack\AppData\Local\Temp\jieba.cache
I0802 13:37:09.727056 15972 __init__.py:131] Loading model from cache C:\Users\jack\AppData\Local\Temp\jieba.cache
Loading model cost 0.780 seconds.
I0802 13:37:10.507883 15972 __init__.py:163] Loading model cost 0.780 seconds.
Prefix dict has been built succesfully.
I0802 13:37:10.508871 15972 __init__.py:164] Prefix dict has been built succesfully.


In [10]:
del stopwords
del split
del temp
gc.collect()

197

In [11]:
splitTitle

['余额 宝 升级 新 基金 收益 流动性',
 '马云 马化腾 这位 部级 副手',
 '慢跑 身体 好处',
 '特斯拉 出车祸 碰撞 起火 二死 一伤 自动 驾驶 安全 保障',
 '李沁 出席 活动 网友 穿 单肩 裙子 美',
 '微信 被动 引流 几个 方法 — — 干货',
 '凯萨琳丽塔琼丝 宝贝 千金   美女 星 二代 卡莉丝',
 '上联 樱花 灿烂 日本 岛 下联',
 '魂斗罗 水下 关 谣言 传 起来',
 '何为 经商 之道 读懂 方可 为商',
 '成都 必去 旅游景点 推荐',
 '识别 新旧 手机',
 '上联 关公 斩华雄 全仗 一壶 老酒 下联',
 '后来 知道 款 进口车 4S店 便宜',
 '2018 沧州 公务员 面试 计划 组织 题 点睛 开展 扶贫 励志 班',
 '上联 : 苦菜花 迎春花 山 菊花 三花 竞放 下联',
 '大佬 朋友圈 背后 短 视频 之争 微视 正用 抖音 方式 进攻 抖音',
 '贵州 旅游 陕西 推介会 暨 贵州 旅游 进校园 活动   启动 仪式 隆重举行',
 '没有 核弹 国家 最有 可能 统一 全球',
 '战神 总监 单机游戏 不会 死   竞争 类 游戏 无感',
 '上市 公司业绩 整体 经济 结构 持续 优化',
 '武夷 水仙 茶   茶香 四溢 仙气 之美',
 '有没有 温暖 言情小说 推荐',
 '清华大学 首次 夺得 全球 高校 计算机科学 排行榜 冠军',
 '旅游 带 镜头 比较',
 '现实 印度 想像 真的 相差 太远',
 '谢霆锋 王菲 张柏芝 罕见 同台 这是 不可 重现 经典',
 '优步 致死 事件 初步 调查结果   决策 软件 行动 及时',
 '王祖贤近 照曝光 身穿 连衣裙 头戴 草帽 体型 似 少女 脸部 老态 尽显',
 '麦迪森 控股 ( 08057 ) 订立 正式 协议   拟 斥资 11.31 亿港元 收购 三只 小熊',
 '今年 大火 几款 霸气 钥匙扣 耐 刮 抗磨损 成功 男士 爱',
 '新疆 男篮 官宣 主教练 李秋平 离任',
 '中国 唯一 太极 占星 村 600 年前 刘伯温 亲手 设计 怪事 不断 未解',
 '美景 百玫 农业 园',
 '斗鱼 tv 亏损 亿   盈利 成 难题 实力 不足   上市 成

In [12]:
TaggededDocument = gensim.models.doc2vec.TaggedDocument

In [13]:
# 把splitTitle改變成所需要的輸入格式，gensim裡的Doc2vec的輸入為固定格式 [句子,句子序號] ，需要用TaggedDocument來包裝輸入的句子
train = []
for i, text in enumerate(splitTitle):    # enumerate是把格式變成 [(0, 'Spring'), (1, 'Summer')]
    word_list = text.split(' ')
    l = len(word_list)
    word_list[l-1] = word_list[l-1].strip()
    document = TaggededDocument(word_list, tags = [i])
    train.append(document)

In [14]:
train

[TaggedDocument(words=['余额', '宝', '升级', '新', '基金', '收益', '流动性'], tags=[0]),
 TaggedDocument(words=['马云', '马化腾', '这位', '部级', '副手'], tags=[1]),
 TaggedDocument(words=['慢跑', '身体', '好处'], tags=[2]),
 TaggedDocument(words=['特斯拉', '出车祸', '碰撞', '起火', '二死', '一伤', '自动', '驾驶', '安全', '保障'], tags=[3]),
 TaggedDocument(words=['李沁', '出席', '活动', '网友', '穿', '单肩', '裙子', '美'], tags=[4]),
 TaggedDocument(words=['微信', '被动', '引流', '几个', '方法', '—', '—', '干货'], tags=[5]),
 TaggedDocument(words=['凯萨琳丽塔琼丝', '宝贝', '千金', '', '', '美女', '星', '二代', '卡莉丝'], tags=[6]),
 TaggedDocument(words=['上联', '樱花', '灿烂', '日本', '岛', '下联'], tags=[7]),
 TaggedDocument(words=['魂斗罗', '水下', '关', '谣言', '传', '起来'], tags=[8]),
 TaggedDocument(words=['何为', '经商', '之道', '读懂', '方可', '为商'], tags=[9]),
 TaggedDocument(words=['成都', '必去', '旅游景点', '推荐'], tags=[10]),
 TaggedDocument(words=['识别', '新旧', '手机'], tags=[11]),
 TaggedDocument(words=['上联', '关公', '斩华雄', '全仗', '一壶', '老酒', '下联'], tags=[12]),
 TaggedDocument(words=['后来', '知道', '款', '进口车', '4S

In [15]:
model = doc2vec.Doc2Vec(train, min_count=1, window=3, size=300, sample=1e-3, nagative=5, workers=4)
model.train(train, total_examples=model.corpus_count, epochs=10)

D:\Anaconda\lib\site-packages\gensim\models\doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
W0802 13:40:24.190883 15972 base_any2vec.py:1182] Effective 'alpha' higher than previous training cycles


In [16]:
model

In [17]:
len(model[0])

300

In [18]:
train_arrays = np.zeros((len(trainData), 300))

In [19]:
for i in range(len(trainData)):
    train_arrays[i] = model.infer_vector([splitTitle[i]])

In [20]:
train_arrays.shape

(382688, 300)

In [21]:
train_arrays = train_arrays[:,np.newaxis,:]
print(train_arrays.shape)

(382688, 1, 300)


In [22]:
# 對dataset label 做編碼
le = LabelEncoder()
new_label = le.fit_transform(trainData.label).reshape(-1,1)

In [23]:
new_label.shape

(382688, 1)

In [24]:
# label list
new_label = new_label.tolist()

In [25]:
#要先轉乘np.array
new_label = np.array(new_label, dtype='float64')
# label encoding
new_label = np_utils.to_categorical(new_label,15)
print(new_label.shape)

(382688, 15)


In [26]:
new_label[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [27]:
# 随机选择20%作为测试集，剩余作为训练集
X_train, X_test, y_train, y_test = train_test_split(train_arrays, new_label, test_size=0.20, random_state=0)

In [28]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(306150, 1, 300)
(76538, 1, 300)
(306150, 15)
(76538, 15)


In [30]:
model = Sequential()

# model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(128, input_shape=(1, X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(32))
model.add(Dropout(0.2))

model.add(Dense(15, activation='softmax'))

model.summary()
model.compile(loss="categorical_crossentropy",optimizer='rmsprop',metrics=["accuracy"])

model_fit = model.fit(X_train,y_train,batch_size=100,epochs=10,
                      validation_data=(X_test,y_test),
#                      callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)] ## 當val-loss不再提升時停止訓練
                     )

W0802 14:05:36.078454 15972 deprecation.py:323] From D:\Anaconda\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 1, 128)            219648    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 128)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                20608     
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 15)                495       
Total params: 240,751
Trainable params: 240,751
Non-trainable params: 0
_________________________________________________________________
Train on 306150 samples, validate on 76538 samples
Epoch 1/10
306150/306150 [==============================] - 22s

In [31]:
test_pred = model.predict(X_test)

In [32]:
test_pred[0]

array([0.01555901, 0.06672046, 0.10488083, 0.09775414, 0.07145392,
       0.04439246, 0.09716398, 0.06691197, 0.1236978 , 0.05947268,
       0.05809033, 0.06448518, 0.00045163, 0.05109   , 0.07787568],
      dtype=float32)

In [33]:
result = model.evaluate(X_train, y_train)
print ('\ntrain Acc:', result[1])

306150/306150 [==============================] - 14s 46us/sample - loss: 2.5176 - accuracy: 0.1344

train Acc: 0.13439164


In [34]:
result = model.evaluate(X_test, y_test)
print ('\ntest Acc:', result[1])

76538/76538 [==============================] - 4s 46us/sample - loss: 2.5201 - accuracy: 0.1353

test Acc: 0.13533147
